<a href="https://colab.research.google.com/github/mohammadgh76/Federated-Learning/blob/main/Flower_Federated_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 11.7 MB/s eta 0:00:00


In [3]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.1.0+cu118 and Flower 1.6.0


In [4]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

100%|██████████| 170498071/170498071 [00:03<00:00, 43341980.14it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


In [5]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [6]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [7]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-11-29 11:09:12,208 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-11-29 11:09:14,712	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-11-29 11:09:18,109 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3933928243.0, 'CPU': 2.0, 'memory': 7867856487.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3933928243.0, 'CPU': 2.0, 'memory': 7867856487.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-11-29 11:09:18,124 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html


(DefaultActor pid=1451) [Client 1] fit, config: {}
(DefaultActor pid=1451) Epoch 1: train loss 0.0638229250907898, accuracy 0.2528888888888889
(DefaultActor pid=1451) [Client 5] fit, config: {}
(DefaultActor pid=1451) Epoch 1: train loss 0.06438450515270233, accuracy 0.23155555555555554
(DefaultActor pid=1451) [Client 9] fit, config: {}


DEBUG flwr 2023-11-29 11:09:36,320 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-29 11:09:36,341 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-11-29 11:09:36,348 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


(DefaultActor pid=1451) Epoch 1: train loss 0.06452934443950653, accuracy 0.236
(DefaultActor pid=1451) [Client 3] evaluate, config: {}
(DefaultActor pid=1451) [Client 4] evaluate, config: {}


DEBUG flwr 2023-11-29 11:09:40,638 | server.py:187 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-29 11:09:40,642 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-11-29 11:09:40,644 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-11-29 11:10:02,599 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-11-29 11:10:02,624 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(DefaultActor pid=1451) Epoch 1: train loss 0.0519898496568203, accuracy 0.38822222222222225
(DefaultActor pid=1451) [Client 0] evaluate, config: {}
(DefaultActor pid=1451) [Client 8] evaluate, config: {}


DEBUG flwr 2023-11-29 11:10:08,558 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-11-29 11:10:08,561 | server.py:153 | FL finished in 50.33451021299999
INFO:flwr:FL finished in 50.33451021299999
INFO flwr 2023-11-29 11:10:08,563 | app.py:226 | app_fit: losses_distributed [(1, 0.061104728857676184), (2, 0.053842438062032066), (3, 0.05193756834665934)]
INFO:flwr:app_fit: losses_distributed [(1, 0.061104728857676184), (2, 0.053842438062032066), (3, 0.05193756834665934)]
INFO flwr 2023-11-29 11:10:08,565 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-29 11:10:08,567 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-11-29 11:10:08,569 | app.py:229 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-11-29 11:10:08,571 | app.py:230 | app_f

(DefaultActor pid=1451) [Client 1] evaluate, config: {}


History (loss, distributed):
	round 1: 0.061104728857676184
	round 2: 0.053842438062032066
	round 3: 0.05193756834665934

In [8]:
# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-11-29 11:14:11,713 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-11-29 11:14:15,806	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-11-29 11:14:17,823 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3930340147.0, 'memory': 7860680295.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3930340147.0, 'memory': 7860680295.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-11-29 11:14:17,826 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html


(DefaultActor pid=2911) [Client 6] fit, config: {}
(DefaultActor pid=2911) Epoch 1: train loss 0.06659378111362457, accuracy 0.2091111111111111
(DefaultActor pid=2911) [Client 3] fit, config: {}
(DefaultActor pid=2911) Epoch 1: train loss 0.06633343547582626, accuracy 0.20622222222222222
(DefaultActor pid=2911) [Client 8] fit, config: {}


DEBUG flwr 2023-11-29 11:14:33,315 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-29 11:14:33,333 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-11-29 11:14:33,343 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


(DefaultActor pid=2911) Epoch 1: train loss 0.06661941856145859, accuracy 0.218
(DefaultActor pid=2911) [Client 4] evaluate, config: {}
(DefaultActor pid=2911) [Client 9] evaluate, config: {}


DEBUG flwr 2023-11-29 11:14:37,836 | server.py:187 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-29 11:14:37,841 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-11-29 11:14:37,844 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-11-29 11:15:01,476 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-11-29 11:15:01,496 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(DefaultActor pid=2911) Epoch 1: train loss 0.09023310989141464, accuracy 0.126
(DefaultActor pid=2911) [Client 0] evaluate, config: {}
(DefaultActor pid=2911) [Client 7] evaluate, config: {}


DEBUG flwr 2023-11-29 11:15:05,728 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-11-29 11:15:05,735 | server.py:153 | FL finished in 47.827416782
INFO:flwr:FL finished in 47.827416782
INFO flwr 2023-11-29 11:15:05,739 | app.py:226 | app_fit: losses_distributed [(1, 3.7060653686523444), (2, 0.35297648175557456), (3, 0.07454340124130249)]
INFO:flwr:app_fit: losses_distributed [(1, 3.7060653686523444), (2, 0.35297648175557456), (3, 0.07454340124130249)]
INFO flwr 2023-11-29 11:15:05,745 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-29 11:15:05,747 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-11-29 11:15:05,750 | app.py:229 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-11-29 11:15:05,752 | app.py:230 | app_fit: metrics_cent

(DefaultActor pid=2911) [Client 2] evaluate, config: {}


History (loss, distributed):
	round 1: 3.7060653686523444
	round 2: 0.35297648175557456
	round 3: 0.07454340124130249

In [9]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [10]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-11-29 11:19:27,036 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-11-29 11:19:31,662	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-11-29 11:19:35,063 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3927814963.0, 'memory': 7855629927.0, 'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3927814963.0, 'memory': 7855629927.0, 'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-11-29 11:19:35,068 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html


Server-side evaluation loss 0.07372879934310914 / accuracy 0.084


(pid=4413) 2023-11-29 11:19:36.916168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=4413) 2023-11-29 11:19:36.916247: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=4413) 2023-11-29 11:19:36.916282: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=4413) 2023-11-29 11:19:38.516051: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(DefaultActor pid=4413) [Client 6] fit, config: {}
(DefaultActor pid=4413) Epoch 1: train loss 0.06437573581933975, accuracy 0.23755555555555555
(DefaultActor pid=4413) [Client 8] fit, config: {}
(DefaultActor pid=4413) Epoch 1: train loss 0.06323998421430588, accuracy 0.24644444444444444
(DefaultActor pid=4413) [Client 7] fit, config: {}


DEBUG flwr 2023-11-29 11:19:51,125 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-29 11:19:51,139 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-11-29 11:19:51,278 | server.py:125 | fit progress: (1, 0.060867159843444825, {'accuracy': 0.304}, 15.42918861999999)
INFO:flwr:fit progress: (1, 0.060867159843444825, {'accuracy': 0.304}, 15.42918861999999)
DEBUG flwr 2023-11-29 11:19:51,281 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-11-29 11:20:19,518 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
INFO flwr 2023-11-29 11:20:19,726 | server.py:125 | fit progress: (3, 0.05194331932067871, {'accuracy': 0.402}, 43.87732743599997)
INFO:flwr:fit progress: (3, 0.05194331932067871, {'accuracy': 0.402}, 43.87732743599997)
DEBUG flwr 2023-11-29 11:20:19,730 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(DefaultActor pid=4413) Epoch 1: train loss 0.052734438329935074, accuracy 0.37933333333333336
Server-side evaluation loss 0.05194331932067871 / accuracy 0.402
(DefaultActor pid=4413) [Client 3] evaluate, config: {}
(DefaultActor pid=4413) [Client 5] evaluate, config: {}


DEBUG flwr 2023-11-29 11:20:24,170 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-11-29 11:20:24,173 | server.py:153 | FL finished in 48.32419801899994
INFO:flwr:FL finished in 48.32419801899994
INFO flwr 2023-11-29 11:20:24,174 | app.py:226 | app_fit: losses_distributed [(1, 0.060401045640309646), (2, 0.052991231918334956), (3, 0.05016530259450277)]
INFO:flwr:app_fit: losses_distributed [(1, 0.060401045640309646), (2, 0.052991231918334956), (3, 0.05016530259450277)]
INFO flwr 2023-11-29 11:20:24,179 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-29 11:20:24,182 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-11-29 11:20:24,185 | app.py:229 | app_fit: losses_centralized [(0, 0.07372879934310914), (1, 0.060867159843444825), (2, 0.05448196887969971), (3, 0.051943319320

(DefaultActor pid=4413) [Client 4] evaluate, config: {}


History (loss, distributed):
	round 1: 0.060401045640309646
	round 2: 0.052991231918334956
	round 3: 0.05016530259450277
History (loss, centralized):
	round 0: 0.07372879934310914
	round 1: 0.060867159843444825
	round 2: 0.05448196887969971
	round 3: 0.05194331932067871
History (metrics, centralized):
{'accuracy': [(0, 0.084), (1, 0.304), (2, 0.36), (3, 0.402)]}

In [11]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [12]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

In [13]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-11-29 11:29:17,483 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-11-29 11:29:21,823	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-11-29 11:29:23,984 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'memory': 7852587419.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'CPU': 2.0, 'object_store_memory': 3926293708.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'memory': 7852587419.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'CPU': 2.0, 'object_store_memory': 3926293708.0}
INFO flwr 2023-11-29 11:29:23,990 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html


Server-side evaluation loss 0.07364963865280151 / accuracy 0.112


(pid=7064) 2023-11-29 11:29:25.753463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=7064) 2023-11-29 11:29:25.753553: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=7064) 2023-11-29 11:29:25.753589: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=7064) 2023-11-29 11:29:27.918501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(DefaultActor pid=7064) [Client 6, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=7064) Epoch 1: train loss 0.06581372022628784, accuracy 0.21688888888888888
(DefaultActor pid=7064) [Client 3, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=7064) Epoch 1: train loss 0.06598072499036789, accuracy 0.21733333333333332
(DefaultActor pid=7064) [Client 8, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flwr 2023-11-29 11:29:39,770 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-29 11:29:39,784 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-11-29 11:29:39,920 | server.py:125 | fit progress: (1, 0.06318190956115723, {'accuracy': 0.302}, 15.549144093999985)
INFO:flwr:fit progress: (1, 0.06318190956115723, {'accuracy': 0.302}, 15.549144093999985)
DEBUG flwr 2023-11-29 11:29:39,925 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-11-29 11:30:17,733 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
INFO flwr 2023-11-29 11:30:17,882 | server.py:125 | fit progress: (3, 0.04914418864250183, {'accuracy': 0.434}, 53.51136350699994)
INFO:flwr:fit progress: (3, 0.04914418864250183, {'accuracy': 0.434}, 53.51136350699994)
DEBUG flwr 2023-11-29 11:30:17,887 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(DefaultActor pid=7064) Epoch 2: train loss 0.04782240092754364, accuracy 0.4431111111111111
Server-side evaluation loss 0.04914418864250183 / accuracy 0.434
(DefaultActor pid=7064) [Client 3] evaluate, config: {}
(DefaultActor pid=7064) [Client 6] evaluate, config: {}


DEBUG flwr 2023-11-29 11:30:22,085 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-11-29 11:30:22,090 | server.py:153 | FL finished in 57.719113147999906
INFO:flwr:FL finished in 57.719113147999906
INFO flwr 2023-11-29 11:30:22,096 | app.py:226 | app_fit: losses_distributed [(1, 0.06300141445795694), (2, 0.052101890484491986), (3, 0.049056919654210406)]
INFO:flwr:app_fit: losses_distributed [(1, 0.06300141445795694), (2, 0.052101890484491986), (3, 0.049056919654210406)]
INFO flwr 2023-11-29 11:30:22,101 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-29 11:30:22,109 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-11-29 11:30:22,114 | app.py:229 | app_fit: losses_centralized [(0, 0.07364963865280151), (1, 0.06318190956115723), (2, 0.05278866124153137), (3, 0.04914418864

(DefaultActor pid=7064) [Client 8] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06300141445795694
	round 2: 0.052101890484491986
	round 3: 0.049056919654210406
History (loss, centralized):
	round 0: 0.07364963865280151
	round 1: 0.06318190956115723
	round 2: 0.05278866124153137
	round 3: 0.04914418864250183
History (metrics, centralized):
{'accuracy': [(0, 0.112), (1, 0.302), (2, 0.39), (3, 0.434)]}

In [14]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-11-29 11:34:04,685 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-11-29 11:34:08,991	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-11-29 11:34:12,479 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'GPU': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3925042790.0, 'memory': 7850085582.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'GPU': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3925042790.0, 'memory': 7850085582.0, 'CPU': 2.0}
INFO flwr 2023-11-29 11:34:12,487 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html


(DefaultActor pid=8468) [Client 551, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=8468) Epoch 1: train loss 0.10265698283910751, accuracy 0.1111111111111111
(DefaultActor pid=8468) Epoch 2: train loss 0.10207810252904892, accuracy 0.13333333333333333
(DefaultActor pid=8468) Epoch 3: train loss 0.1010790765285492, accuracy 0.15555555555555556
(DefaultActor pid=8468) [Client 815, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=8468) Epoch 1: train loss 0.10238944739103317, accuracy 0.1111111111111111
(DefaultActor pid=8468) Epoch 2: train loss 0.10155917704105377, accuracy 0.1111111111111111
(DefaultActor pid=8468) Epoch 3: train loss 0.10058996826410294, accuracy 0.1111111111111111
(DefaultActor pid=8468) [Client 956, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=8468) Epoch 1: train loss 0.10236511379480362, accuracy 0.1111111111111111
(DefaultActor pid=8468) Epoch 2: train loss 0.10190302

DEBUG flwr 2023-11-29 11:35:00,602 | server.py:236 | fit_round 1 received 25 results and 0 failures
DEBUG:flwr:fit_round 1 received 25 results and 0 failures
WARNING flwr 2023-11-29 11:35:00,654 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-11-29 11:35:00,658 | server.py:173 | evaluate_round 1: strategy sampled 50 clients (out of 1000)
DEBUG:flwr:evaluate_round 1: strategy sampled 50 clients (out of 1000)


(DefaultActor pid=8468) [Client 912, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=8468) Epoch 1: train loss 0.10237844288349152, accuracy 0.15555555555555556
(DefaultActor pid=8468) Epoch 2: train loss 0.1013769879937172, accuracy 0.15555555555555556
(DefaultActor pid=8468) Epoch 3: train loss 0.10036572813987732, accuracy 0.15555555555555556
(DefaultActor pid=8468) [Client 643] evaluate, config: {}
(DefaultActor pid=8468) [Client 836] evaluate, config: {}
(DefaultActor pid=8468) [Client 526] evaluate, config: {}
(DefaultActor pid=8468) [Client 38] evaluate, config: {}
(DefaultActor pid=8468) [Client 112] evaluate, config: {}
(DefaultActor pid=8468) [Client 139] evaluate, config: {}
(DefaultActor pid=8468) [Client 737] evaluate, config: {}
(DefaultActor pid=8468) [Client 185] evaluate, config: {}
(DefaultActor pid=8468) [Client 87] evaluate, config: {}
(DefaultActor pid=8468) [Client 149] evaluate, config: {}
(DefaultActor pid=8468) [Client 45] evaluat

DEBUG flwr 2023-11-29 11:36:19,762 | server.py:187 | evaluate_round 1 received 50 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 50 results and 0 failures
WARNING flwr 2023-11-29 11:36:19,767 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-11-29 11:36:19,773 | server.py:222 | fit_round 2: strategy sampled 25 clients (out of 1000)


...

DEBUG flwr 2023-11-29 11:38:59,333 | server.py:236 | fit_round 3 received 25 results and 0 failures
DEBUG:flwr:fit_round 3 received 25 results and 0 failures
DEBUG flwr 2023-11-29 11:38:59,387 | server.py:173 | evaluate_round 3: strategy sampled 50 clients (out of 1000)
DEBUG:flwr:evaluate_round 3: strategy sampled 50 clients (out of 1000)


(DefaultActor pid=8468) Epoch 1: train loss 0.1006162017583847, accuracy 0.13333333333333333
(DefaultActor pid=8468) Epoch 2: train loss 0.10014144331216812, accuracy 0.15555555555555556
(DefaultActor pid=8468) Epoch 3: train loss 0.09947309643030167, accuracy 0.2
(DefaultActor pid=8468) [Client 177] evaluate, config: {}
(DefaultActor pid=8468) [Client 585] evaluate, config: {}
(DefaultActor pid=8468) [Client 270] evaluate, config: {}
(DefaultActor pid=8468) [Client 471] evaluate, config: {}
(DefaultActor pid=8468) [Client 273] evaluate, config: {}
(DefaultActor pid=8468) [Client 392] evaluate, config: {}
(DefaultActor pid=8468) [Client 61] evaluate, config: {}
(DefaultActor pid=8468) [Client 798] evaluate, config: {}
(DefaultActor pid=8468) [Client 643] evaluate, config: {}
(DefaultActor pid=8468) [Client 802] evaluate, config: {}
(DefaultActor pid=8468) [Client 133] evaluate, config: {}
(DefaultActor pid=8468) [Client 788] evaluate, config: {}
(DefaultActor pid=8468) [Client 951] eva

DEBUG flwr 2023-11-29 11:40:18,412 | server.py:187 | evaluate_round 3 received 50 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 50 results and 0 failures
INFO flwr 2023-11-29 11:40:18,415 | server.py:153 | FL finished in 365.82960991999994
INFO:flwr:FL finished in 365.82960991999994
INFO flwr 2023-11-29 11:40:18,417 | app.py:226 | app_fit: losses_distributed [(1, 0.4597048997879027), (2, 0.4561514654159546), (3, 0.45142192745208737)]
INFO:flwr:app_fit: losses_distributed [(1, 0.4597048997879027), (2, 0.4561514654159546), (3, 0.45142192745208737)]
INFO flwr 2023-11-29 11:40:18,421 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-29 11:40:18,424 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-11-29 11:40:18,426 | app.py:229 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-11-29 11:40:18,429 | app.py:230 | app_fit: 

(DefaultActor pid=8468) [Client 426] evaluate, config: {}


History (loss, distributed):
	round 1: 0.4597048997879027
	round 2: 0.4561514654159546
	round 3: 0.45142192745208737